# Реализация частотного конвертора

In [57]:
import numpy as np
import scipy
from scipy.fft import fft, ifft
from scipy.signal import fftconvolve
import IPython.display as ipd
from IPython import display
import librosa
from librosa import display

**Задача:**
- реализовать преобразование, сопоставляющее входному дискретному сигналу *x*, с частотой дискретизации *N_1*, выходной сигнал *y* с частотой дискретизации *N_2*

**Алгоритм:**
- растяжение сигнала
- фильтрация сигнала
- прореживание

## V1

Реализуем алгоритм последовательно в функции `frequency_converter_by_fft`

In [95]:
def frequency_converter_by_fft(audio, sr, target_sr):
    
    # Растяжение
    n = target_sr * len(audio)
    x_hidden, x_hidden[np.arange(n) % target_sr == 0] =  np.zeros(n), audio
    
    # Фильтрация в частотной области с помощью теоремы о свертке
    h, h[:target_sr] = np.zeros(n), sr
    y = ifft(fft(x_hidden) * h)
    y = np.around(np.abs(y))
    
    # Прореживание
    y = y[::sr]
    
    return y

Однако благодаря подсказке моего одногруппика - *Александра Зубакова*, было замечено, что в данном алгоритме можно упростить операцию свертки. В блоке кода ниже можно увидеть исходный массив после растяжения, а также после использования операции свертки.

In [89]:
n_ = 10
t_sr = 4
sr = 5
x = np.random.randint(6, size=n_)
print('Исходный массив: ', x)
n = t_sr * len(x)
x_, x_[np.arange(n) % t_sr == 0] =  np.zeros(n), x
print("Массив после растяжения: ", x_)
print("Массив после фильтрации: ", np.convolve(x_, np.ones(t_sr)))
print("Массив после фильтрации в частотной области: ", np.around(np.abs(fftconvolve(x_, np.ones(t_sr)))))

Исходный массив:  [0 0 2 4 3 3 0 2 3 2]
Массив после растяжения:  [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 4. 0. 0. 0. 3. 0. 0. 0. 3. 0. 0. 0.
 0. 0. 0. 0. 2. 0. 0. 0. 3. 0. 0. 0. 2. 0. 0. 0.]
Массив после фильтрации:  [0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 4. 4. 4. 4. 3. 3. 3. 3. 3. 3. 3. 3.
 0. 0. 0. 0. 2. 2. 2. 2. 3. 3. 3. 3. 2. 2. 2. 2. 0. 0. 0.]
Массив после фильтрации в частотной области:  [0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 4. 4. 4. 4. 3. 3. 3. 3. 3. 3. 3. 3.
 0. 0. 0. 0. 2. 2. 2. 2. 3. 3. 3. 3. 2. 2. 2. 2. 0. 0. 0.]


Таким образом, можно `переписать` алгоритм в более *компактном* виде, повысив его *производительность*:

## V2

In [3]:
def frequency_converter_opt(audio, sr, target_sr):
    return np.repeat(audio, target_sr)[::sr]

## Результат работы

Загрузим речь Ленина и выполним преобразование с помощью частотного конвертера:

In [10]:
audio, sr = librosa.load('resources/lenin-obraschenie.wav', duration=20)

In [11]:
# Исходная частота дискретизации
print(sr)

22050


In [12]:
ipd.Audio(audio, rate=sr)

In [14]:
# Частота дискретизации после преобразования
target_sr = 16_000

In [15]:
%%time
y_ = frequency_converter_opt(audio, sr, target_sr)

CPU times: total: 32.5 s
Wall time: 32.8 s


In [16]:
ipd.Audio(y_, rate=target_sr)